<a href="https://colab.research.google.com/github/Habibcool/Harga-saham-pada-tesla/blob/main/UAS_4571_BDDM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## B.preprocessing sampai dengan metric evaluation.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

data = pd.read_csv('/content/TESLA.csv')

print
display(data.head(10))

,Unnamed: 0,Date,Open,High,Low,Close,Adj Close,Volume
0,0,6/29/10,1.266667,1.666667,1.169333,1.592667,1.592667,281494500
1,1,6/30/10,1.719333,2.028000,1.553333,1.588667,1.588667,257806500
2,2,7/1/10,1.666667,1.728000,1.351333,1.464000,1.464000,123282000
3,3,7/2/10,1.533333,1.540000,1.247333,1.280000,1.280000,77097000
4,4,7/6/10,1.333333,1.333333,1.055333,1.074000,1.074000,103003500
5,5,7/7/10,1.093333,1.108667,0.998667,1.053333,1.053333,103825500
6,6,7/8/10,1.076000,1.168000,1.038000,1.164000,1.164000,115671000
7,7,7/9/10,1.172000,1.193333,1.103333,1.160000,1.160000,60759000
8,8,7/12/10,1.196667,1.204667,1.133333,1.136667,1.136667,33037500
9,9,7/13/10,1.159333,1.242667,1.126667,1.209333,1.209333,40201500


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

data = pd.read_csv('/content/sample_data/TESLA.csv')

# Menghapus nilai yang hilang
data.dropna(inplace=True)

In [ ]:
from sklearn.preprocessing import StandardScaler

# Normalisasi harga penutupan
scaler = StandardScaler()
data[['Close']] = scaler.fit_transform(data[['Close']])


In [ ]:
# Membagi data menjadi set pelatihan dan pengujian, dengan rasio 80:20.
train_size = int(len(data) * 0.8)
train_data = data[:train_size]
test_data = data[train_size:]


In [ ]:
import numpy as np

def create_dataset(dataset, n_steps=1):
    X, y = [], []
    for i in range(len(dataset) - n_steps - 1):
        X.append(dataset[i:(i + n_steps), 0])
        y.append(dataset[i + n_steps, 0])
    return np.array(X), np.array(y)

n_steps = 10  # jumlah langkah waktu
X_train, y_train = create_dataset(train_data[['Close']].values, n_steps)
X_test, y_test = create_dataset(test_data[['Close']].values, n_steps)

# Reshape input untuk LSTM [samples, time steps, features]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))


In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(n_steps, 1)))
model.add(LSTM(50))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=4)


Epoch 1/10
725/725 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - loss: 0.0014
Epoch 2/10
725/725 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - loss: 0.0019
Epoch 3/10
725/725 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - loss: 0.0014
Epoch 4/10
725/725 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - loss: 0.0018
Epoch 5/10
725/725 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - loss: 0.0022
Epoch 6/10
725/725 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - loss: 0.0017
Epoch 7/10
725/725 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - loss: 0.0020
Epoch 8/10
725/725 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - loss: 0.0014
Epoch 9/10
725/725 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - loss: 0.0012
Epoch 10/10
725/725 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - loss: 0.0015


In [ ]:
# Mengevaluasi performanya menggunakan metrik RMSE dan MAPE.
from sklearn.metrics import mean_squared_error

# Prediksi pada data uji
y_pred = model.predict(X_test)

# Menghitung RMSE dan MAPE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

print(f'RMSE: {rmse}')
print(f'MAPE: {mape}%')


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
RMSE: 0.08494040374128298
MAPE: 47.51302633085082%


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import pandas as pd

# Memuat data
data = pd.read_csv('/content/sample_data/TESLA.csv')
# Mengubah kolom 'Date' ke tipe data datetime dan mengekstrak fitur numerik
data['Date'] = pd.to_datetime(data['Date'])  # Mengubah 'Date' ke datetime
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Day'] = data['Date'].dt.day

# Memisahkan fitur dan target
X = data.drop(columns=['High', 'Date'])  # Menghapus kolom 'High' dan 'Date'
y = data['High']

# Mendapatkan feature importance
importances = model.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

print(feature_importance_df)

<ipython-input-5-6204559556af>:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['Date'] = pd.to_datetime(data['Date'])  # Mengubah 'Date' ke datetime


      Feature  Importance
3       Close    0.355953
4   Adj Close    0.303077
2         Low    0.173064
1        Open    0.167304
0  Unnamed: 0    0.000533
5      Volume    0.000033
8         Day    0.000017
7       Month    0.000011
6        Year    0.000008


In [ ]:
# Membagi data menjadi set pelatihan dan pengujian
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Membuat model Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)



RandomForestRegressor(random_state=42)

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mape = mean_absolute_percentage_error(y_test, y_pred)

print(f'RMSE: {rmse}')
print(f'MAPE: {mape}%')
from sklearn.metrics import r2_score

r2 = r2_score(y_test, y_pred)

print(f'R-squared: {r2:.2f}')

RMSE: 1.2527875273550049
MAPE: 0.7489500177530407%
R-squared: 1.00
